In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_name= 'thyroidDF'
df = pd.read_csv(f'data/{dataset_name}.csv')

In [ ]:
print(df.dtypes)
df.head()

In [ ]:
boolean_columns = df.columns[df.isin(['t', 'f']).all()]
df[boolean_columns] = df[boolean_columns].applymap(lambda x: 1 if x == 't' else 0)

df.head()

In [ ]:
df['target'].unique()

In [ ]:
hyperthyroid = {'A', 'B', 'C', 'D'}
hypothyroid = {'E', 'F', 'G', 'H'}

def classify_diagnosis(diagnosis):
    
    letters = set(diagnosis.replace('|', ''))
    if letters & hyperthyroid:
        return 2
    
    if letters & hypothyroid:
        return 0
    
    return 1

df['target'] = df['target'].apply(classify_diagnosis)
df['target'].unique()

In [ ]:
df.drop(['patient_id'],axis='columns', inplace=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

mesures = ['TSH_measured', 'T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured']
valors = ['TSH', 'T3','TT4','T4U','FTI', 'TBG']
df_cleared = df.drop(mesures+valors,axis='columns')
df_cleared = df_cleared.dropna()
numeric_features = df_cleared.select_dtypes(np.number).keys()
non_numeric = [k for k in df_cleared.keys() if k not in numeric_features]
df_cleared.loc[:,non_numeric] = df_cleared.loc[:,non_numeric].apply(LabelEncoder().fit_transform)

correlation_matrix = df_cleared.corr(method='pearson', min_periods=1, numeric_only=False)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5) 
plt.title('Heatmap de la matriu de correlació', fontsize=15)
plt.ylabel('Variables', fontsize = 12)
plt.savefig('correlation_matrix_atributs_no_mesures.png')
plt.show()

In [ ]:
# Distribució de la variable 'Survived'
plt.figure(figsize=(4, 2))
sns.countplot(x='target', data=df, palette='viridis')
plt.title('Distribució de target')
plt.xlabel('Survived (0 = No, 1 = Yes)')
plt.ylabel('Count')
plt.xticks(ticks=[1, 0, 2], labels=['healthy','hypothyroid','hyperthyroid'])
plt.show()

In [ ]:
data_pivot = df.groupby(['referral_source', 'target']).size().unstack(fill_value=0)

data_pivot = data_pivot.div(data_pivot.sum(axis=1), axis=0)

data_pivot.plot(kind='bar', stacked=True, colormap='viridis')
plt.ylabel('Proporció')
plt.title('Distribució de target per referral_source')
plt.savefig("proporcio_referral_source_target.pdf")
plt.show()


In [ ]:
nan_percentage = df.isna().mean()
nan_columns = nan_percentage[nan_percentage > 0]
nan_columns

In [ ]:
df['referral_source'].unique()

In [ ]:
df['referral_source'].value_counts()


In [ ]:
dummies_referral_source = pd.get_dummies(df['referral_source'], dtype='int',drop_first=False)
df = pd.concat([df, dummies_referral_source], axis=1)
df = df.drop(['referral_source','WEST'], axis="columns")

In [ ]:
df.head()

In [ ]:
df['age'].nsmallest(5)

In [ ]:
df['age'].nlargest(5)

In [ ]:
df = df[df['age'] < 100]



In [ ]:
# Omplir NaNs de 'sexe' amb 'F' si 'pregnant' és 1
df.loc[(df['sex'].isna()) & (df['pregnant'] == 1), 'sex'] = 'F'

In [ ]:
dist_nan = df['target'].value_counts(normalize=True).sort_index()
df.dropna(subset=['sex'], inplace=True)
dist_not_nan = df['target'].value_counts(normalize=True).sort_index()

print("Distribució target amb NaNs:")
print(dist_nan)
print("\nDistribució target sense NaNs:")
print(dist_not_nan)

In [ ]:
dummies_sex = pd.get_dummies(df['sex'], dtype='int',drop_first=False)
df = pd.concat([df, dummies_sex], axis=1)
df = df.drop(['sex'], axis="columns")
df.head()

In [ ]:
nan_percentage = df.isna().mean()
nan_columns = nan_percentage[nan_percentage > 0]
nan_columns

In [ ]:
mesures = ['TSH_measured', 'T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured']
valors = ['TSH', 'T3','TT4','T4U','FTI', 'TBG']
for i in range(len(mesures)):
    no_mesured = df[df[mesures[i]] == 0]
    is_theory_correct = no_mesured[valors[i]].isna().all()
    print(is_theory_correct)


In [ ]:
df.drop(mesures,axis='columns',inplace=True)

In [ ]:
df_fill_na = df.copy()
valors_maxims_normals = [5.6,6,150,1.3,165,39]
for i,j in zip(valors,valors_maxims_normals):
    df_fill_na[i]/=j
df_fill_na.head(10)

In [ ]:

# Tornar a crear les gràfiques, aquesta vegada partint del dataframe
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
axes = axes.flatten()

for ax, column in zip(axes, valors):
    ax.hist(df_fill_na[column], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax.set_title(f'Distribució de {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Freqüència')

plt.tight_layout()
plt.show()

In [ ]:
for i in range(len(valors)):
    print(df_fill_na[valors[i]].min(), df_fill_na[valors[i]].max())

In [ ]:
for i in valors:
    std_dev = df[i].std()  
    df_fill_na[i] = df_fill_na[i].fillna(-1)
df_fill_na.head()

In [ ]:
import matplotlib.pyplot as plt

# Tornar a crear les gràfiques, aquesta vegada partint del dataframe
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
axes = axes.flatten()

for ax, column in zip(axes, valors):
    ax.hist(df_fill_na[column], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax.set_title(f'Distribució de {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Freqüència')

plt.tight_layout()
plt.show()

In [ ]:
df_fill_na['age'] /= 100

In [ ]:
df_fill_na.dropna(inplace=True)

In [ ]:
print(df_fill_na['target'].value_counts())

In [ ]:
import seaborn as sns

df_cleared = df[valors+['target']]
correlation_matrix = df_cleared.corr(method='pearson', min_periods=1, numeric_only=False)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix,annot=True, cmap='coolwarm',fmt='.2f', linewidths=0.5) 
plt.title('Heatmap de la matriu de correlació')
plt.ylabel('Variables', fontsize = 12)
plt.savefig('correlation_matrix_atributs_df_fillna.pdf')
plt.show()

In [ ]:
# Crear subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

# Filtrar dades per sexe
women = df_fill_na[df_fill_na['M'] == 0]
men = df_fill_na[df_fill_na['M'] == 1]

# Colors i targets
targets_numeric = [1,0,2]
colors = ['green', 'red', 'blue']
target_categorical=['healthy','hypothyroid','hyperthyroid']

# Subplot per a dones
for n, color,c in zip(targets_numeric, colors,target_categorical):
    sns.histplot(
        data=women[women['target'] == n],
        x="age",
        bins=18,
        label=c,
        ax=axes[0],
        color=color,
        kde=False
    )
axes[0].legend()
axes[0].set_title('Female')

# Subplot per a homes
for n, color,c in zip(targets_numeric, colors,target_categorical):
    sns.histplot(
        data=men[men['target'] == n],
        x="age",
        bins=18,
        label=c,
        ax=axes[1],
        color=color,
        kde=False
    )
axes[1].legend()
axes[1].set_title('Male')

# Mostrar la gràfica
plt.tight_layout()
plt.savefig("relacio_edat_sexe_target.pdf")
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
target_att = 'target'
attributes = [k for k in df.keys() if k!= target_att]
X = df_fill_na[attributes]
y = df_fill_na[[target_att]]
X_train_val_fillna, X_test_fillna, y_train_val_fillna, y_test_fillna = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
y_train_val_fillna.value_counts()

In [ ]:
y_test_fillna.value_counts()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
# Provem PCA amb 2 components
pca = PCA(n_components=2)
y_pca = np.array(y_train_val_fillna)
X_data_pca_2 = pca.fit_transform(X_train_val_fillna)

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

ax.scatter(X_data_pca_2[:,0], X_data_pca_2[:,1],c = y_pca[:], s = 40, cmap='viridis')
ax.grid()
plt.savefig("PCA_df_fillna.pdf")

X_train, X_val, y_train, y_val = train_test_split(X_data_pca_2, y_pca, test_size=0.2, random_state=20)
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
cm = confusion_matrix(y_val, y_pred)

print(f1_score(y_val,y_pred,average='macro'))
print(accuracy_score(y_val,y_pred))
print("Matriu de confusió:\n", cm)
print("Train:",f1_score(y_train,y_pred_train,average='macro'))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_val, y_train, y_val = train_test_split(X_train_val_fillna, y_train_val_fillna, test_size=0.2, random_state=0)
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred = model.predict(X_val)

cm = confusion_matrix(y_val, y_pred)

print(f1_score(y_val,y_pred,average='macro'))
print(accuracy_score(y_val,y_pred))
print("Matriu de confusió:\n", cm)
print("Train:",f1_score(y_train,y_pred_train,average='macro'))




In [ ]:
def classificacio_per_defecte(X, y, PRC, it, base_classifiers):
    acc_r = np.zeros((it, len(base_classifiers)))

    random_state = 20
    for i in range(it):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=PRC, random_state=random_state * (i + 1))
        for j, model in enumerate(base_classifiers.values()):
            model.fit(X_train, y_train)

            yhat = model.predict(X_test)
            acc_r[i][j] = f1_score(y_test, yhat, average='macro')

    return acc_r



In [ ]:
def save_df(acc_r,name_dataset,base_classifiers):
    models = list(base_classifiers.keys())

    data_for_saving = []
    for j in range(acc_r.shape[1]):  
        for i in range(acc_r.shape[0]):  
            precisio = acc_r[i, j]
            
            data_for_saving.append({
                'Model': models[j],
                'Mostra': i + 1,  
                'Precisio': precisio
            })

    df_results = pd.DataFrame(data_for_saving)

    df_results.to_csv(f'./execucions/resultats_per_model_default_{name_dataset}.csv', index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC


random_state=20
base_classifiers = {
    'Logistic Regression OvO': OneVsOneClassifier(LogisticRegression(random_state=random_state)),
    'Logistic Regression OvR': OneVsRestClassifier(LogisticRegression(random_state=random_state)),
    'SVC OvO': OneVsOneClassifier(SVC(random_state=random_state)),
    'SVC OvR': OneVsRestClassifier(SVC(random_state=random_state)),
    'Decision Tree': DecisionTreeClassifier(random_state=random_state),
    'Random Forest': RandomForestClassifier(random_state=random_state),
    'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'XGBoost': XGBClassifier(random_state=random_state)
}
PRC = 0.3
#f1_dfna = classificacio_per_defecte(X_train_val_fillna,y_train_val_fillna,PRC,10,base_classifiers)
#save_df(f1_dfna,'df_fillna',base_classifiers)
f1_dfna = pd.read_csv('./execucions/resultats_per_model_default_df_fillna.csv')

In [ ]:
def plot_data(acc_r):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=acc_r, x="Model", y="Precisio")
    plt.title("Comparativa de f1_score per Model")
    plt.ylabel("f1_score")
    plt.xlabel("Model")
    plt.legend(title="Tipus")
    plt.show()

In [ ]:
plot_data(f1_dfna)

In [ ]:

from sklearn.feature_selection import SelectFromModel


selected_data = {}

def select_important_features(model, X_train, y_train, threshold='mean'):
    model.fit(X_train, y_train)
    if hasattr(model, 'feature_importances_'):
        selector = SelectFromModel(model, threshold=threshold, prefit=True)
        mask = selector.get_support()
    elif hasattr(model, 'coef_'):
        selector = SelectFromModel(model, threshold=threshold, prefit=True)
        mask = selector.get_support()
    else:
        print(f" --- El model {model.__class__.__name__} no suporta selecció de característiques. S'afageixen totes les variables---")
        mask = np.ones(X_train.shape[1], dtype=bool)
    X_norm_selected = X_train[:, mask] if isinstance(X_train, np.ndarray) else X_train.loc[:, mask]
    return X_norm_selected, mask


for name, model in base_classifiers.items():
    print(f"Seleccionant característiques importants per {name}")
    X_norm_selected, mask = select_important_features(model, X_train_val_fillna, y_train_val_fillna)
    selected_data[f'{name}'] = X_norm_selected
    selected_data[f'{name}_mask'] = mask

In [ ]:

classifier_names = base_classifiers.keys()
feature_selection_df = pd.DataFrame(
    index=classifier_names, 
    columns=attributes
)
for model_name in classifier_names:
    mask = selected_data[f'{model_name}_mask']
    feature_selection_df.loc[model_name] = mask

feature_selection_df = feature_selection_df.astype(int)

plt.figure(figsize=(12, 8))
sns.heatmap(feature_selection_df, cmap='Blues', annot=True, cbar=False, 
            linewidths=.5, linecolor='black')
plt.title('Característiques Seleccionades per Model', fontsize=16)
plt.xlabel('Característiques', fontsize=12)
plt.ylabel('Models', fontsize=12)

plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

In [ ]:
selected_features_models = {
    'Decision Tree': DecisionTreeClassifier(random_state=random_state),
    'Random Forest': RandomForestClassifier(random_state=random_state),
    'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
    'XGBoost': XGBClassifier(random_state=random_state)
}
it = 10
acc_r = np.zeros((it, len(selected_features_models)))

random_state = 20
for i in range(it):
    j=0
    for name, model in selected_features_models.items():
        X_train, X_test, y_train, y_test = train_test_split(selected_data[name], y_train_val_fillna, test_size=PRC, random_state=random_state * (i + 1))
        model.fit(X_train, y_train)

        yhat = model.predict(X_test)
        acc_r[i][j] = f1_score(y_test, yhat, average='macro')
        j += 1 


In [ ]:
#save_df(acc_r,'selected_features_df_fillna',selected_features_models)
s_f = pd.read_csv('./execucions/resultats_per_model_default_selected_features_df_fillna.csv')
plot_data(s_f)

In [ ]:
# Paràmetres pel Logistic Regressor
lr_param_grid = {
    'estimator__random_state' : [random_state],  
    'estimator__penalty' : ['l2', 'l1'], 
    'estimator__solver' : ['lbfgs', 'liblinear'],  
    'estimator__C' : [0.01, 0.1, 1.0, 10.0, 100.0],
}

svc_param_grid = {'estimator__random_state' : [random_state],
                  'estimator__C' : [0.5, 1.0, 2.0, 4.0],
                  'estimator__gamma' : ['auto'],
                  'estimator__kernel' : ['rbf', 'linear', 'poly', 'sigmoid'],
                  'estimator__shrinking' : [True, False]}


# Paràmetres pel DTC
dtc_param_grid = {'random_state' : [random_state],
                  'criterion' : ['gini'],
                  'max_depth' : [None, 3, 5, 7, 10],
                  'min_samples_leaf' : [1, 5, 10], 
                  'min_samples_split' : [2, 5, 10],
                  'max_features': [None, 1, 2, 3, 4]}

# Paràmetres pel RFC
rfc_param_grid = {'random_state': [random_state],
                  'criterion': ['gini', 'entropy'],
                  'max_features': [None, 2, 3, 4],
                  'max_depth': [None, 10, 20, 30],
                  'min_samples_split': [2, 5, 10],
                  'min_samples_leaf': [1, 2, 4],
                  'class_weight': [None, 'balanced'],
}

# Paràmetres pel GBC
gbc_param_grid = {
    'random_state': [random_state],
    'loss': ['log_loss'], 
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
}



# Paràmetres pel KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 10, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [30, 40, 50],
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
}

# Paràmetres pel XGBoost
xgb_param_grid = {
    'random_state': [random_state],
    'n_estimators': [100, 200, 300],  
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  
    'max_depth': [3, 5, 7, 10], 
    'min_child_weight': [1, 3, 5],  
    'subsample': [0.6, 0.8, 1.0],  
}



In [ ]:
param_grid_list = {
    'Logistic Regression OvO': lr_param_grid, 
    'Logistic Regression OvR': lr_param_grid, 
    'SVC OvO': svc_param_grid,
    'SVC OvR': svc_param_grid,
    'Decision Tree': dtc_param_grid, 
    'Random Forest': rfc_param_grid, 
    'Gradient Boosting': gbc_param_grid,
    'KNN': knn_param_grid,
    'XGBoost': xgb_param_grid
}

In [ ]:
import time
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer

def cerca_hyperparametres(X,y,base_classifiers,splits,dataset_name,param_grid_list):
    random_state = 42

    kfold = StratifiedKFold(n_splits=splits, random_state=random_state, shuffle=True)

    for name, model in base_classifiers.items():
        results = []

        start_time = time.time()

        param_grid = param_grid_list[name]

        gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, 
                            scoring=make_scorer(f1_score, average='macro'), n_jobs=-1)
        gs.fit(X, y)
        mean_f1_best_model = gs.best_score_
        best_params = gs.best_params_


        end_time = time.time()
        calc_time = end_time - start_time

        results.append({
            'Classifier': name,
            'Mean Test Score (F1)': mean_f1_best_model,
            'Execution Time (s)': calc_time,  
            'Best Parameters': best_params,
            'Best Estimator': gs.best_estimator_  
        })


        print(f'Classifier: {type(model).__name__}, Mean Test Score: {mean_f1_best_model:.4f}, Time: {calc_time:.2f}s')

        results_df = pd.DataFrame(results)
        results_df.to_csv(f'./execucions/{name}_hyperparameters_search_results_{dataset_name}.csv', index=False)


In [ ]:
base_classifiers_df_fillna = {
    'Logistic Regression OvO': OneVsOneClassifier(LogisticRegression(random_state=random_state)),
    'Logistic Regression OvR': OneVsRestClassifier(LogisticRegression(random_state=random_state)),
    'Decision Tree': DecisionTreeClassifier(random_state=random_state),
    'Random Forest': RandomForestClassifier(random_state=random_state),
    'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'XGBoost': XGBClassifier(random_state=random_state)
}
#cerca_hyperparametres(X_train_val_fillna,y_train_val_fillna,base_classifiers_df_fillna,10,'df_fillna',param_grid_list)

In [ ]:
for name, model in selected_features_models.items():
    cerca_hyperparametres(selected_data[name],y_train_val_fillna,{name: model},10,'selected_features_df_fillna',param_grid_list)

In [ ]:
def get_best_model_cerca_hyperparametres(base_classifiers,dataset_name):
    best_model=LogisticRegression()
    f1=0
    for name, model in base_classifiers.items():
        l=pd.read_csv(f'./execucions/{name}_hyperparameters_search_results_{dataset_name}.csv')
        l = l.loc[l['Mean Test Score (F1)'].idxmax()]
        if f1 < l['Mean Test Score (F1)']:
            f1 = l['Mean Test Score (F1)']
            best_params = eval(l['Best Parameters'])
            best_model = model
            best_model.set_params(**best_params)
    return best_model


In [ ]:
best_model_dfna = get_best_model_cerca_hyperparametres(base_classifiers_df_fillna,'df_fillna')
best_model_dfna

In [ ]:
def passar_a_intervals(df,hormones,intervals):
    def classificar_h(h,interval):
        if pd.isna(h):  
            return [0, 0, 0]
        elif h < interval[0]:
            return [1, 0, 0]
        elif h < interval[1]:
            return [0, 1, 0]
        else:
            return [0, 0, 1]


    for hormona,interval in zip(hormones,intervals):
        df[[f'{hormona} Baix', f'{hormona} Normal', f'{hormona} Alt']] = df[hormona].apply(lambda x: pd.Series(classificar_h(x,interval)))
        df.drop([hormona],axis='columns',inplace=True)
    return df

intervals=[[0.26,5.6],
            [1.8,4.6],
            [60,150],
            [0.7,1.3],
            [60,165],
            [13,39]
]
df_intervals = passar_a_intervals(df.copy(),valors,intervals)

df_intervals.head()

In [ ]:

bins = [0,20,35,50,65,80,100]
labels=['0-19','20-34','35-49','50-64','65-79','80-99']
df_intervals['Grup_Edat'] = pd.cut(df_intervals['age'], bins=bins, labels=labels, right=False)
dummies_sex = pd.get_dummies(df_intervals['Grup_Edat'], dtype='int',drop_first=True)
df_intervals = pd.concat([df_intervals, dummies_sex], axis=1)

df_intervals = df_intervals.drop(['age','Grup_Edat'], axis="columns")

In [ ]:
vars = [[f'{i} Alt',f'{i} Baix',f'{i} Normal'] for i in valors]
vars = [j for a in vars for j in a]
df_cleared = df_intervals[vars+['target']+labels[1:]]
correlation_matrix = df_cleared.corr(method='pearson', min_periods=1, numeric_only=False)

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5) 
plt.title('Heatmap de la matriu de correlació')
plt.savefig('correlation_matrix_atributs_df_intervals.pdf')
plt.show()

In [ ]:
attributes = [k for k in df_intervals.keys() if k!= target_att]
X = df_intervals[attributes]
y = df_intervals[[target_att]]
X_train_val_intervals, X_test_intervals, y_train_val_intervals, y_test_intervals = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
pca = PCA(n_components=2)
y_pca = np.array(y_train_val_intervals)
X_data_pca_2 = pca.fit_transform(X_train_val_intervals)

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

ax.scatter(X_data_pca_2[:,0], X_data_pca_2[:,1],c = y_pca[:], s = 40, cmap='viridis')
ax.grid()
plt.savefig("PCA_df_intervals.pdf")

X_train, X_val, y_train, y_val = train_test_split(X_data_pca_2, y_train_val_intervals, test_size=0.2, random_state=20)
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
cm = confusion_matrix(y_val, y_pred)

print(f1_score(y_val,y_pred,average='macro'))
print(accuracy_score(y_val,y_pred))
print("Matriu de confusió:\n", cm)
print("Train:",f1_score(y_train,y_pred_train,average='macro'))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val_intervals, y_train_val_intervals, test_size=0.2, random_state=20)
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
cm = confusion_matrix(y_val, y_pred)

print(f1_score(y_val,y_pred,average='macro'))
print(accuracy_score(y_val,y_pred))
print("Matriu de confusió:\n", cm)
print("Train:",f1_score(y_train,y_pred_train,average='macro'))

In [ ]:
PRC = 0.3
#f1_intervals = classificacio_per_defecte(X_train_val_intervals,y_train_val_intervals,PRC,10,base_classifiers)
#save_df(f1_intervals,'df_interval',base_classifiers)
f1_intervals = pd.read_csv('./execucions/resultats_per_model_default_df_interval.csv')

In [ ]:
plot_data(f1_intervals)

In [ ]:
#cerca_hyperparametres(X_train_val_intervals,y_train_val_intervals,base_classifiers,10,'df_intervals',param_grid_list)

In [ ]:
import os

# Inicialitzem els diccionaris per a gestionar els valors
scores = {
    "df_fillna": {},
    "df_intervals": {},
    "selected_features_df_fillna": {},
    "selected_features_df_intervals": {}
}

# Llista dels tipus de fitxers
file_categories = [
    ("df_fillna", "_hyperparameters_search_results_df_fillna.csv"),
    ("df_intervals", "_hyperparameters_search_results_df_intervals.csv"),
    ("selected_features_df_fillna", "_hyperparameters_search_results_selected_features_df_fillna.csv"),
    ("selected_features_df_intervals", "_hyperparameters_search_results_selected_features_df_intervals.csv")
]

# Llegim els fitxers disponibles i extreiem els valors de 'Mean Test Score (F1)'
for model_name in base_classifiers.keys():
    # Iterem sobre les categories de fitxers
    for category, suffix in file_categories:
        file_path = f"execucions/{model_name}{suffix}"

        # Comprovar si el fitxer existeix i llegir la columna 'Mean Test Score (F1)'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            scores[category][model_name] = df['Mean Test Score (F1)'].iloc[0]
        else:
            scores[category][model_name] = None

# Filtrar models amb almenys un valor vàlid en qualsevol dels fitxers
valid_models = [
    model for model in base_classifiers.keys()
    if any(scores[category].get(model) is not None for category in scores)
]

# Preparar valors per a la gràfica
x = range(len(valid_models))
values = {category: [scores[category].get(model, 0) if scores[category].get(model) is not None else 0 for model in valid_models]
          for category in scores}

# Configuració per a la gràfica
bar_width = 0.2
offsets = [-1.5 * bar_width, -0.5 * bar_width, 0.5 * bar_width, 1.5 * bar_width]
colors = ['skyblue', 'orange', 'green', 'purple']
labels = [
    'df_fillna',
    'df_intervals',
    'selected_features_df_fillna',
    'selected_features_df_intervals'
]

plt.figure(figsize=(14, 8))

# Dibuixem les barres per a cada categoria
for i, (category, vals) in enumerate(values.items()):
    plt.bar([pos + offsets[i] for pos in x], vals, bar_width, label=labels[i], color=colors[i])

# Configuració dels eixos i estètica
plt.xlabel('Model', fontsize=12)
plt.ylabel('Mean Test Score (F1)', fontsize=12)
plt.title('Comparació de Mean Test Score (F1) per models i tipus de fitxer', fontsize=14)
plt.xticks(x, valid_models, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, class_names=None, figsize=(8, 6)):
    # Calcular la matriu de confusió
    cm = confusion_matrix(y_true, y_pred)
    
    # Crear una matriu de colors
    cm_colored = np.full_like(cm, fill_value=0.5, dtype=float)  # Valor per defecte blanc (1.0)
    
    # Assignar colors: Verd per la diagonal i Vermell per la resta
    for i in range(len(cm)):
        for j in range(len(cm)):
            if cm[i, j] != 0:
                if i == j:
                    cm_colored[i, j] = 0.7  # Verd per la diagonal
                else:
                    cm_colored[i, j] = 0.1  # Vermell per la resta

    # Crear la imatge de la matriu de confusió
    plt.figure(figsize=figsize)
    plt.imshow(cm_colored, cmap='RdYlGn', interpolation='nearest', vmin=0, vmax=1)
    
    # Afegir els valors sobre la matriu
    for i in range(len(cm)):
        for j in range(len(cm)):
            plt.text(j, i, cm[i, j], ha='center', va='center', 
                     color='black' if cm[i, j] != 0 else 'white', fontsize=12)
    
    # Afegir títols i etiquetes
    plt.title("Matriu de Confusió", fontsize=15)
    plt.xlabel('Etiqueta Predicha', fontsize=12)
    plt.ylabel('Etiqueta Real', fontsize=12)
    
    # Ajustar les etiquetes dels eixos si es donen
    plt.xticks(np.arange(len(class_names)), class_names, rotation=45)
    plt.yticks(np.arange(len(class_names)), class_names)

    # Mostrar la matriu
    plt.show()


In [ ]:
best_model_dfna.fit(X_train_val_fillna,y_train_val_fillna)
y_pred = best_model_dfna.predict(X_test_fillna)
plot_confusion_matrix(y_test_fillna, y_pred,['hipotiroidisme','sa','hipertiroidisme'])

print(f1_score(y_test_fillna,y_pred,average='macro'))
print(f1_score(y_test_fillna,y_pred,average='micro'))
print(accuracy_score(y_test_fillna,y_pred))
print("Matriu de confusió:\n", cm)

In [ ]:
best_model_intervals = get_best_model_cerca_hyperparametres(base_classifiers,'df_intervals')
best_model_intervals

In [ ]:
best_model_intervals.fit(X_train_val_intervals,y_train_val_intervals)
y_pred = best_model_intervals.predict(X_test_intervals)

cm = confusion_matrix(y_test_intervals, y_pred)

print(f1_score(y_test_intervals,y_pred,average='macro'))
print(f1_score(y_test_intervals,y_pred,average='micro'))
print(accuracy_score(y_test_intervals,y_pred))
print("Matriu de confusió:\n", cm)

In [ ]:
from sklearn.feature_selection import SelectFromModel


selected_data = {}

def select_important_features(model, X_train, y_train, threshold='mean'):
    model.fit(X_train, y_train)
    if hasattr(model, 'feature_importances_'):
        selector = SelectFromModel(model, threshold=threshold, prefit=True)
        mask = selector.get_support()
    elif hasattr(model, 'coef_'):
        selector = SelectFromModel(model, threshold=threshold, prefit=True)
        mask = selector.get_support()
    else:
        print(f" --- El model {model.__class__.__name__} no suporta selecció de característiques. S'afageixen totes les variables---")
        mask = np.ones(X_train.shape[1], dtype=bool)
    X_norm_selected = X_train[:, mask] if isinstance(X_train, np.ndarray) else X_train.loc[:, mask]
    return X_norm_selected, mask


for name, model in base_classifiers.items():
    print(f"Seleccionant característiques importants per {name}")
    X_norm_selected, mask = select_important_features(model, X_train_val_intervals, y_train_val_intervals)
    selected_data[f'{name}'] = X_norm_selected
    selected_data[f'{name}_mask'] = mask

In [ ]:

classifier_names = base_classifiers.keys()
feature_selection_df = pd.DataFrame(
    index=classifier_names, 
    columns=attributes
)
for model_name in classifier_names:
    mask = selected_data[f'{model_name}_mask']
    feature_selection_df.loc[model_name] = mask

feature_selection_df = feature_selection_df.astype(int)

plt.figure(figsize=(12, 8))
sns.heatmap(feature_selection_df, cmap='Blues', annot=True, cbar=False, 
            linewidths=.5, linecolor='black')
plt.title('Característiques Seleccionades per Model', fontsize=16)
plt.xlabel('Característiques', fontsize=12)
plt.ylabel('Models', fontsize=12)

plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

In [ ]:
selected_features_models = {
    'Decision Tree': DecisionTreeClassifier(random_state=random_state),
    'Random Forest': RandomForestClassifier(random_state=random_state),
    'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
    'XGBoost': XGBClassifier(random_state=random_state)
}
it = 10
acc_r = np.zeros((it, len(selected_features_models)))

random_state = 20
for i in range(it):
    j=0
    for name, model in selected_features_models.items():
        X_train, X_test, y_train, y_test = train_test_split(selected_data[name], y_train_val_intervals, test_size=PRC, random_state=random_state * (i + 1))
        model.fit(X_train, y_train)

        yhat = model.predict(X_test)
        acc_r[i][j] = f1_score(y_test, yhat, average='macro')
        j += 1 


In [ ]:
save_df(acc_r,'selected_features_df_intervals',selected_features_models)
s_f = pd.read_csv('./execucions/resultats_per_model_default_selected_features_df_intervals.csv')
plot_data(s_f)

In [ ]:
selected_features_models = {
    'XGBoost': XGBClassifier(random_state=random_state)
}
#for name, model in selected_features_models.items():
    #cerca_hyperparametres(selected_data[name],y_train_val_intervals,{name: model},10,'selected_features_df_intervals',param_grid_list)